In [1]:
import json
import re
from collections import Counter

dataset_path = r"D:\AI-Inosuke\data\dataset.jsonl"

# Load dataset
data = []
with open(dataset_path, "r", encoding="utf-8") as f:
    for line in f:
        line = line.strip()
        if line:
            try:
                data.append(json.loads(line))
            except json.JSONDecodeError:
                print("JSON lỗi ở dòng:", line)

print(f"\nTổng số mẫu: {len(data)}")

# Validate schema
bad_samples = []
for i, ex in enumerate(data):
    if not all(k in ex for k in ["instruction", "input", "output"]):
        bad_samples.append(i)
    if not all(isinstance(ex[k], str) for k in ["instruction", "input", "output"]):
        bad_samples.append(i)

if bad_samples:
    print(f"{len(bad_samples)} mẫu bị lỗi schema (index: {bad_samples[:10]} ...)")
else:
    print("Tất cả mẫu có schema hợp lệ (instruction, input, output là string)")

# Length statistics
len_instr = [len(ex["instruction"]) for ex in data]
len_input = [len(ex["input"]) for ex in data]
len_output = [len(ex["output"]) for ex in data]

print(f"Trung bình instruction dài {sum(len_instr)/len(len_instr):.1f} ký tự")
print(f"Trung bình input dài {sum(len_input)/len(len_input):.1f} ký tự")
print(f"Trung bình output dài {sum(len_output)/len(len_output):.1f} ký tự")

# Check duplicate instructions
instr_counts = Counter(ex["instruction"] for ex in data)
dupes = [k for k, v in instr_counts.items() if v > 1]
print(f"Số instruction trùng: {len(dupes)}")
if dupes:
    print("   Ví dụ:", dupes[:5])

# Check multi-turn dialogues
multi_turn = 0
for ex in data:
    text = ex["input"] + ex["output"]
    if re.search(r"Bạn:|Người khác:", text):
        if text.count("\n") >= 2:
            multi_turn += 1
one_turn = len(data) - multi_turn
print(f"1-turn: {one_turn} mẫu ({one_turn/len(data)*100:.1f}%)")
print(f"Multi-turn: {multi_turn} mẫu ({multi_turn/len(data)*100:.1f}%)")

# Role distribution in dialogues
roles = Counter()
for ex in data:
    text = ex["input"] + ex["output"]
    matches = re.findall(r"(Bạn:|Người khác:)", text)
    roles.update(matches)

print("Phân bố vai trò hội thoại:")
for role, count in roles.most_common():
    print(f"   {role}: {count}")





Tổng số mẫu: 9259
Tất cả mẫu có schema hợp lệ (instruction, input, output là string)
Trung bình instruction dài 42.5 ký tự
Trung bình input dài 9.5 ký tự
Trung bình output dài 74.6 ký tự
Số instruction trùng: 125
   Ví dụ: ['Đoạn hội thoại nhiều lượt về quạ.', 'Đoạn hội thoại nhiều lượt về Sanemi.', 'Nói một câu đậm chất Inosuke (dài).', 'Miêu tả bản thân như Inosuke.', 'Đoạn hội thoại nhiều lượt về Gyutaro.']
1-turn: 7708 mẫu (83.2%)
Multi-turn: 1551 mẫu (16.8%)
Phân bố vai trò hội thoại:
   Người khác:: 1850
   Bạn:: 1252
